In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
dis = 'drive/My Drive/data_final/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
d_train = pd.read_csv(dis + 'transactions_train.csv')
d_test = pd.read_csv(dis + 'transactions_test.csv')

d_train['is_train'] = 1
d_test['is_train'] = 0

data = pd.concat([d_train, d_test], ignore_index=True)

col = list(data.columns)
col.insert(0, col[-1])
col.pop(-1)
data = data[col]

In [ ]:
data.to_csv(dis + 'data1.csv', index = False)

In [ ]:
data = pd.read_csv(dis + 'data1.csv')

In [ ]:
def Fill(name, cat_counts_train):
  for i in range(199):
    if name+str(i) not in cat_counts_train.columns:
      cat_counts_train[name+str(i)] = 0

  cat_counts_train = cat_counts_train[[name+str(i) for i in range(199)]]


In [ ]:
def Preprocess(data0, l, flag):
  #Aggregation & count small group
  data = data0.loc[data0['trans_date'] < l]

  agg_features = data.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()

  counter_df_train = data.groupby(['client_dk','small_group'])['amount'].count()
  cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
  cat_counts_train = cat_counts_train.fillna(0)
  cat_counts_train.columns = ['small_group_'+str(i) for i in cat_counts_train.columns]

  Fill('small_group_', cat_counts_train)

  for i in cat_counts_train.columns:
    cat_counts_train[i] = cat_counts_train[i].astype(int)

  train_target = pd.read_csv(dis + 'train_target.csv')
  train_target['is_train'] = 1

  train_target = train_target.merge(agg_features, on = 'client_dk', how = 'outer').fillna(0)

  for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_target[str(i)] = train_target[str(i)].astype(int)
 
  train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

  train_target['is_train'] = train_target['is_train'].astype(int)

  col = list(train_target.columns)
  col.insert(0, col[15])
  col.pop(16)
  train_target = train_target[col]  

  #Small group sum
  counter_df_train = data.groupby(['client_dk','small_group'])['amount'].sum()
  cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
  cat_counts_train = cat_counts_train.fillna(0)
  cat_counts_train.columns = ['small_group_sum_'+str(i) for i in cat_counts_train.columns]

  Fill('small_group_sum_', cat_counts_train)

  train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

  #Small group mean
  counter_df_train = data.groupby(['client_dk','small_group'])['amount'].mean()
  cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
  cat_counts_train = cat_counts_train.fillna(0)
  cat_counts_train.columns = ['small_group_mean_'+str(i) for i in cat_counts_train.columns]

  Fill('small_group_mean_', cat_counts_train)

  train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

  #Small group std
  counter_df_train = data.groupby(['client_dk','small_group'])['amount'].std()
  cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
  cat_counts_train = cat_counts_train.fillna(0)
  cat_counts_train.columns = ['small_group_std_'+str(i) for i in cat_counts_train.columns]

  Fill('small_group_std_', cat_counts_train)

  train_target = train_target.merge(cat_counts_train, how = 'outer', on = 'client_dk')

  #Small group rate
  for i in range(199):
    if 'small_group_' + str(i) in train_target.columns:
      train_target['small_group_rate_' + str(i)] = train_target['small_group_' + str(i)] / train_target['count']

  train_target = train_target.drop(['count'], axis = 1)

  train_target = train_target.set_index('client_dk')

  #New Target
  if flag:
    td = data0.loc[(data0['trans_date'] >= l) & (data0['trans_date'] < l + 7)]

    counter_df_train = td.groupby(['client_dk','small_group'])['amount'].count()
    cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    cat_counts_train = cat_counts_train.fillna(0)
    cat_counts_train.columns = [i for i in cat_counts_train.columns]

    for i in cat_counts_train.columns:
      cat_counts_train[i] = cat_counts_train[i].astype(int)

    for i in [27, 32,	41,	45,	67,	73,	81,	88]:
      if i in cat_counts_train.columns:
        train_target[str(i)] = (cat_counts_train[i] > 0).astype(int)
      else:
        train_target[str(i)] = 0

  train_target = train_target.fillna(0)

  for i in [27, 32,	41,	45,	67,	73,	81,	88]:
    train_target[str(i)] = train_target[str(i)].astype(int)

  #
  

  return train_target
  

In [ ]:
d = []
for i in range(1, 4):
  d.append(Preprocess(data, i*100, True))

d.append(Preprocess(data, 4*100, False))

In [ ]:
for i in range(4):
  d[0].to_csv(dis + 'new_data/predata3_' + str(i+1) + '.csv')

In [ ]:
def GetWeekly(data, l):
    td = data.loc[(data['trans_date'] >= l) & (data['trans_date'] < l + 7)]

    counter_df_train = td.groupby(['client_dk','small_group'])['amount'].count()
    cat_counts_train = counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    cat_counts_train = cat_counts_train.fillna(0)
    cat_counts_train.columns = [i for i in cat_counts_train.columns]

    for i in cat_counts_train.columns:
      cat_counts_train[i] = cat_counts_train[i].astype(int)

    return cat_counts_train[[27, 32,	41,	45,	67,	73,	81,	88]]

## Favor small group

In [ ]:
t = Preprocess(data, 400, False)

In [ ]:
def FindFavor(x):
  a = list(zip(list(x), [i for i in range(204)]))
  a = sorted(a)[::-1]
  return [a[i][1] for i in range(10)]

b = t[['small_group_sum_' + str(i) for i in range(199)]].apply(FindFavor, axis = 1)

In [ ]:
dc = {}

for i in range(10):
  dc[i] = []
  for j in b.index:
    dc[i].append(b[j][i])

In [ ]:
from keras.utils import to_categorical

for i in range(10):
  dc[i] = to_categorical(dc[i], 199)

Using TensorFlow backend.


In [ ]:
for i in range(10):
  for j in range(199):
    t['favor_small_group_' + str(i) + '_' + str(j)] = dc[i][:,j]
    t['favor_small_group_' + str(i) + '_' + str(j)] = t['favor_small_group_' + str(i) + '_' + str(j)].astype(int)

In [ ]:
t.to_csv(dis + 'predata4.csv')

## Delta Small Group

In [ ]:
import progressbar
import time
bar = progressbar.ProgressBar().start() # Создаём новый progress bar

d = [[] for i in range(199)]
for i in range(199):
  c = data.loc[data['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
  for j in range(50000):
    if j in c.index:
      c0 = list(c[j].index)
      if len(c0) > 1:
        d[i].append((c0[-1] - c0[0])/(len(c0) - 1))
      else:
        d[i].append(387)
    else:
      d[i].append(-1)
  bar.update((i + 1)) 

| |     #                                           | 199 Elapsed Time: 0:16:40

In [ ]:
for i in range(199):
  t['dalta_day_small_group_' + str(i)] = d[i]

In [ ]:
t.to_csv(dis + 'predata5.csv')

##Last buy

In [ ]:
t = pd.read_csv(dis + 'predata5.csv', index_col = 'client_dk')

In [ ]:
data.loc[data['small_group'] == 27].groupby(['client_dk', 'trans_date'])['amount'].sum()

client_dk  trans_date
1          31             513.323
           173            507.592
           286           1145.708
2          59              26.588
           111             19.337
                           ...   
49999      184              5.133
           202              9.668
           216             15.400
           371             15.400
           386             15.470
Name: amount, Length: 202885, dtype: float64

In [ ]:
import progressbar
import time
bar = progressbar.ProgressBar().start() # Создаём новый progress bar

d = [[] for i in range(199)]
for i in range(199):
  c = data.loc[data['small_group'] == i].groupby(['client_dk', 'trans_date'])['amount'].sum()
  for j in range(50000):
    if j in c.index:
      c0 = list(c[j].index)
      d[i].append((387 - c0[-1]))
    else:
      d[i].append(-1)
  bar.update((i + 1)) 

| |                               #                 | 199 Elapsed Time: 0:16:46

In [ ]:
for i in range(199):
  t['last_buy_small_group_' + str(i)] = d[i]

[[112,
  23,
  49,
  8,
  1,
  36,
  63,
  35,
  110,
  2,
  76,
  114,
  104,
  89,
  46,
  33,
  53,
  123,
  83,
  29,
  25,
  340,
  121,
  10,
  95,
  127,
  77,
  69,
  5,
  144,
  1,
  41,
  75,
  96,
  27,
  9,
  67,
  105,
  137,
  14,
  7,
  42,
  1,
  68,
  51,
  1,
  6,
  67,
  69,
  360,
  6,
  29,
  17,
  2,
  16,
  0,
  32,
  4,
  15,
  207,
  72,
  54,
  157,
  17,
  63,
  0,
  264,
  23,
  8,
  126,
  11,
  4,
  1,
  90,
  148,
  123,
  57,
  235,
  10,
  2,
  1,
  14,
  154,
  20,
  16,
  4,
  -1,
  270,
  87,
  2,
  66,
  95,
  52,
  53,
  21,
  93,
  85,
  0,
  4,
  4,
  57,
  2,
  166,
  21,
  0,
  22,
  24,
  78,
  87,
  150,
  10,
  56,
  128,
  72,
  11,
  21,
  51,
  90,
  11,
  190,
  0,
  107,
  100,
  114,
  109,
  11,
  54,
  349,
  -1,
  156,
  0,
  29,
  48,
  167,
  142,
  188,
  54,
  2,
  115,
  12,
  141,
  56,
  25,
  170,
  125,
  51,
  23,
  16,
  22,
  126,
  37,
  108,
  60,
  157,
  88,
  58,
  13,
  129,
  131,
  11,
  166,
  122,
  77,
  16,
 